# 如何实现缠论？

## 合并K线

这一步比较简单：

1. 有一个序列，内容是合并后的K线（merged_candle）。其初始值是第一根普通K线。
2. 逐根进行普通K线与 merged_candle 中最后一个已合并的K线（merged_candle[-1]）进行比较。如有包含则向前再取一根已合并的K线（merged_candle[-2]），比较 merged_candle[-2] 和 merged_candle[-1] 的关系，决定合并方向。
3. 循环至结束。

## 分型

缠论原文对分型的要求是：

1. 合并完成的K线；

2. 不同方向的3根。


合并完成的K线——我称之为缠论K线，非合并的称之为普通K线以示区别——保证了相邻的缠论K线之间，比较价位（无论是比较最高价还是最低价）都只有 > 和 < 两种，去除了 = 这种可能。

因此：

1. 2根缠论K线之间有 $(2-1=1)$ 次比较、2种关系，一共有 $2^{(2-1)}=2$ 种可能；

2. 3根缠论K线之间有 $(3-1=2)$ 次比较、2种关系，一共有 $2^{(3-1)}=4$ 种可能；

3. 4根缠论K线之间有 $(4-1=3)$ 次比较、2种关系，一共有 $2^{(4-1)}=8$ 种可能；

    .

    .

    .

结论：

1. $n$ 根缠论K线之间有 $(n-1)$ 次比较、2种关系，一共有 $2^{(n-1)}$ 种可能；

2. 这些可能性中，有一半是镜像相似的，如下图：

![镜像相似](./pictures/mirror.png)

## 设想1

缠论的分型必须是3根缠论K线。于是很多情况下通达信缠论插件之类的工具，当没有办法确定最后一笔（当前笔？）是否完成时，不绘制最后一笔。这令人很迷茫。

因此有了对缠论分型再次分类、扩充的想法：

降低缠论分型的要求，从而增加两个分型，一个用于起点绘制，另一个用于终点绘制。如图：

![非完整分型](./pictures/uncompleted_fractal.png)

解释一下：

1. 一个完整的缠论分型包含左、中、右3根缠论K线，而起点分型只有中、右2根，同样终点分型只有左、中2根。

2. 起点分型适用于第1根缠论K线（比如新上市的股票）或者跳空情况。这种情况下搜寻分型的左边那个K线似乎没有意义。

3. 终点分型适用于最后1根缠论K线，比如收盘的情况。这种情况下我们假设再开盘的K线就是标准（完整）缠论分型的右侧那根。

## 分型的确认、修正与丢弃

### 确认

在以下两种情况下，可以确认之前的分型：

1. 当前缠论K线与前一个分型的顶或底之间的距离 >= 4，可以确认前分型有效。

    ![确认](./pictures/confirm_1.png)

    如上图，无论顶分型是在下一根K线成立还是更后面成立，此时K线数量已足够多，因此可以确认底分型有效。

2. 顶底分型之间有一个或以上的分型

    ……是不是多余？

### 2021-11-04 修正

遇到被连续丢弃两次的情况。

![连续丢弃分型的情况](./pictures/2021-11-03_02-39_al2112.png)

因此还是需要 is_confirmed 这个变量，用来控制 f-2 不被丢弃。

找到问题，迅速修好。

![修复连续丢弃分型的情况](./pictures/2021-11-04_11-54_al2112.png)

AL真是好朋友，提供各种极端情况。回头要专门跑全AL的图像。

## 笔

从绘图的角度，笔很简单，就是连接分型就好。

或许把笔作为一个数据结构，能提供一下笔的时间周期、价格幅度以及斜率什么的？

## 线段

1. 如果笔的数量小于3，略过。

2. 笔的数量大于等于3：

    A. 如果还没有线段，取最近的3个笔，尝试是否能生成线段。

    B. 如果已经有了线段，跳过一个与线段异向的笔，取紧接着的同向笔，判断笔的右侧值是否超越线段的右侧值。如果超越，线段延伸；否则线段终结。

如何判断能否生成线段，缠论的定义是三笔有重叠部分，则有线段。那么，三笔主要是四个分型，如下：

![三笔四点](./pictures/strokes_and_fractals.png)

此图有几种变形，关键在于 F-2 与 F-3、F-4 的关系。